# Text generation using RNN - Character Level (TO BE RUN IN GOOGLE COLAB) 

To generate text using RNN, we need a to convert raw text to a supervised learning problem format.

Take, for example, the following corpus:

"Her brother shook his head incredulously"

First we need to divide the data into tabular format containing input (X) and output (y) sequences. In case of a character level model, the X and y will look like this:

|      X     |  Y  |
|------------|-----|
|    Her b   |  r  |
|    er br   |  o  |
|    r bro   |  t  |
|     brot   |  h  |
|    broth   |  e  |
|    .....   |  .  |
|    .....   |  .  |
|    ulous   |  l  |
|    lousl   |  y  |

Note that in the above problem, the sequence length of X is five characters and that of y is one character. Hence, this is a many-to-one architecture. We can, however, change the number of input characters to any number of characters depending on the type of problem.

A model is trained on such data. To generate text, we simply give the model any five characters using which it predicts the next character. Then it appends the predicted character to the input sequence (on the extreme right of the sequence) and discards the first character (character on extreme left of the sequence). Then it predicts again using the new sequence and the cycle continues until a fix number of iterations. An example is shown below:

Seed text: "incre"

|      X                                            |  Y                       |
|---------------------------------------------------|--------------------------|
|                        incre                      |    < predicted char 1 >  |
|               ncre < predicted char 1 >              |    < predicted char 2 >  |
|       cre< predicted char 1 > < predicted char 2 >   |    < predicted char 3 >  |
|       re< predicted char 1 >< predicted char 2 > < predicted char 3 >   |    < predicted char 4 >  |
|                      ...                          |            ...           |

# Notebook Overview
1. Preprocess data
2. LSTM model
3. Generate code

In [1]:
# import libraries
import warnings
warnings.filterwarnings("ignore")

import os
import re
import numpy as np
import random
import sys
import io
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import get_file

# 1. Preprocess data

We're going to build a C code generator by training an RNN on a huge corpus of C code (the linux kernel code). You can download the C code used as source text from the following link:
https://github.com/torvalds/linux/tree/master/kernel

We have already downloaded the entire kernel folder and stored in a local directory

## Load C code

In [2]:
# set path where C files reside

print("Current working directory:", os.getcwd())

path = r"linux_kernel"

os.chdir(path)

file_names = os.listdir()
print(file_names)

Current working directory: /content
['compat.c', 'static_call.c', 'exec_domain.c', 'sysctl.c', 'watch_queue.c', 'tracepoint.c', 'pid_sysctl.h', 'Makefile', 'sys.c', 'scs.c', 'utsname_sysctl.c', 'workqueue_internal.h', 'fork.c', 'sys_ni.c', 'seccomp.c', 'Kconfig.kexec', 'padata.c', 'reboot.c', 'iomem.c', 'resource.c', 'tsacct.c', 'audit_fsnotify.c', 'module_signature.c', 'auditsc.c', 'stop_machine.c', 'kallsyms_internal.h', 'exit.h', 'rseq.c', 'user_namespace.c', 'cpu.c', 'kcmp.c', 'kthread.c', 'backtracetest.c', 'elfcorehdr.c', 'taskstats.c', 'acct.c', 'kexec.c', 'bounds.c', 'context_tracking.c', 'vhost_task.c', 'kallsyms_selftest.h', 'kheaders.c', 'exit.c', 'freezer.c', 'signal.c', 'kexec_elf.c', 'dma.c', 'notifier.c', 'watchdog_buddy.c', 'uid16.c', 'audit.h', 'latencytop.c', 'cred.c', 'configs.c', 'kexec_file.c', 'regset.c', 'ksysfs.c', 'user.c', 'crash_core.c', 'jump_label.c', 'Kconfig.preempt', 'fail_function.c', 'watchdog_perf.c', 'pid_namespace.c', 'kallsyms_selftest.c', 'pid.c',

In [3]:
# use regex to filter .c files
import re
c_names = ".*\.c$"

c_files = list()

for file in file_names:
    if re.match(c_names, file):
        c_files.append(file)

print(c_files)

['compat.c', 'static_call.c', 'exec_domain.c', 'sysctl.c', 'watch_queue.c', 'tracepoint.c', 'sys.c', 'scs.c', 'utsname_sysctl.c', 'fork.c', 'sys_ni.c', 'seccomp.c', 'padata.c', 'reboot.c', 'iomem.c', 'resource.c', 'tsacct.c', 'audit_fsnotify.c', 'module_signature.c', 'auditsc.c', 'stop_machine.c', 'rseq.c', 'user_namespace.c', 'cpu.c', 'kcmp.c', 'kthread.c', 'backtracetest.c', 'elfcorehdr.c', 'taskstats.c', 'acct.c', 'kexec.c', 'bounds.c', 'context_tracking.c', 'vhost_task.c', 'kheaders.c', 'exit.c', 'freezer.c', 'signal.c', 'kexec_elf.c', 'dma.c', 'notifier.c', 'watchdog_buddy.c', 'uid16.c', 'latencytop.c', 'cred.c', 'configs.c', 'kexec_file.c', 'regset.c', 'ksysfs.c', 'user.c', 'crash_core.c', 'jump_label.c', 'fail_function.c', 'watchdog_perf.c', 'pid_namespace.c', 'kallsyms_selftest.c', 'pid.c', 'smpboot.c', 'resource_kunit.c', 'stackleak.c', 'params.c', 'utsname.c', 'scftorture.c', 'workqueue.c', 'ucount.c', 'kprobes.c', 'nsproxy.c', 'ptrace.c', 'crash_reserve.c', 'range.c', 'audit

In [4]:
# load all c code in a list
full_code = list()
for file in c_files:
    code = open(file, "r", encoding='utf-8')
    full_code.append(code.read())
    code.close()

In [5]:
# let's look at how a typical C code looks like
print(full_code[20])

// SPDX-License-Identifier: GPL-2.0-or-later
/*
 * kernel/stop_machine.c
 *
 * Copyright (C) 2008, 2005	IBM Corporation.
 * Copyright (C) 2008, 2005	Rusty Russell rusty@rustcorp.com.au
 * Copyright (C) 2010		SUSE Linux Products GmbH
 * Copyright (C) 2010		Tejun Heo <tj@kernel.org>
 */
#include <linux/compiler.h>
#include <linux/completion.h>
#include <linux/cpu.h>
#include <linux/init.h>
#include <linux/kthread.h>
#include <linux/export.h>
#include <linux/percpu.h>
#include <linux/sched.h>
#include <linux/stop_machine.h>
#include <linux/interrupt.h>
#include <linux/kallsyms.h>
#include <linux/smpboot.h>
#include <linux/atomic.h>
#include <linux/nmi.h>
#include <linux/sched/wake_q.h>

/*
 * Structure to determine completion condition and record errors.  May
 * be shared by works on different cpus.
 */
struct cpu_stop_done {
	atomic_t		nr_todo;	/* nr left to execute */
	int			ret;		/* collected return value */
	struct completion	completion;	/* fired if nr_todo reaches 0 */
};

/* the act

In [6]:
# merge different c codes into one big c code
text = "\n".join(full_code)
print("Total number of characters in entire code: {}".format(len(text)))

Total number of characters in entire code: 2242645


In [7]:
# top_n: only consider first top_n characters and discard the rest for memory and computational efficiency
top_n = 400000
text = text[:top_n]

## Convert characters to integers

In [8]:
# create character to index mapping
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [9]:
print("Vocabulary size: {}".format(len(chars)))

Vocabulary size: 97


## Divide data in input (X) and output (y)

### Create sequences

In [10]:
# define length for each sequence
MAX_SEQ_LENGTH = 50          # number of input characters (X) in each sequence
STEP           = 3           # increment between each sequence
VOCAB_SIZE     = len(chars)  # total number of unique characters in dataset

sentences  = []              # X
next_chars = []              # y

for i in range(0, len(text) - MAX_SEQ_LENGTH, STEP):
    sentences.append(text[i: i + MAX_SEQ_LENGTH])
    next_chars.append(text[i + MAX_SEQ_LENGTH])

In [11]:
print('Number of training samples: {}'.format(len(sentences)))

Number of training samples: 133317


## Create input and output using the created sequences

When you're not using the Embedding layer of the Keras as the very first layer, you need to convert your data in the following format:
#### input shape should be of the form :  (#samples, #timesteps, #features)
#### output shape should be of the form :  (#samples, #timesteps, #features)

![Tensor shape](./jupyter resources/rnn_tensor.png)

#samples: the number of data points (or sequences)
#timesteps: It's the length of the sequence of your data (the MAX_SEQ_LENGTH variable).
#features: Number of features depends on the type of problem. In this problem, #features is the vocabulary size, that is, the dimensionality of the one-hot encoding matrix using which each character is being represented. If you're working with **images**, features size will be equal to: (height, width, channels), and the input shape will be (#training_samples, #timesteps, height, width, channels)

In [12]:
# create X and y
X = np.zeros((len(sentences), MAX_SEQ_LENGTH, VOCAB_SIZE), dtype=bool)
y = np.zeros((len(sentences), VOCAB_SIZE), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [13]:
print("Shape of X: {}".format(X.shape))
print("Shape of y: {}".format(y.shape))

Shape of X: (133317, 50, 97)
Shape of y: (133317, 97)


Here, X is reshaped to (#samples, #timesteps, #features). We have explicitly mentioned the third dimension (#features) because we won't use the Embedding() layer of Keras in this case since there are only 97 characters. Characters can be represented as one-hot encoded vector. There are no word embeddings for characters.

# 2. LSTM

In [14]:
from keras.optimizers import Adam

# define model architecture - using a two-layer LSTM with 128 LSTM cells in each layer
model = Sequential()
model.add(LSTM(128, input_shape=(MAX_SEQ_LENGTH, VOCAB_SIZE), return_sequences=True, dropout=0.5))
model.add(LSTM(128, dropout=0.5))
model.add(Dense(VOCAB_SIZE, activation = "softmax"))

optimizer = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics = ['acc'])

In [15]:
# check model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 50, 128)             │         115,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 97)                  │          12,513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 259,809 (1014.88 KB)

 Trainable params: 259,809 (1014.88 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# fit model
model.fit(X, y, batch_size=128, epochs=20)

Epoch 1/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - acc: 0.1795 - loss: 3.2511
Epoch 2/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - acc: 0.1783 - loss: 3.2595
Epoch 3/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - acc: 0.1882 - loss: 3.1969
Epoch 4/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - acc: 0.1934 - loss: 3.1745
Epoch 5/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - acc: 0.2021 - loss: 3.1304
Epoch 6/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - acc: 0.2565 - loss: 2.8483
Epoch 7/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 13s 12ms/step - acc: 0.1845 - loss: 3.2430
Epoch 8/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - acc: 0.1857 - loss: 3.2297
Epoch 9/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - acc: 0.2916 - loss: 2.6705
Epoch 10/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 13s 12ms/step - acc: 0.3087 - loss: 2.5846
Epoch 11/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - acc: 0.3169 - loss: 2.5461
Epoch 12/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - acc:

# 3. Generate code

Create a function that will make next character predictions based on temperature. If temperature is greater than 1, the generated characters will be more versatile and diverse. On the other hand, if temperature is less than one, the generated characters will be much more conservative.

In [18]:
# define function to sample next word from a probability array based on temperature
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [19]:
np.random.multinomial(10, [0.05, 0.9, 0.05], size=2)

array([[ 0,  9,  1],
       [ 0, 10,  0]])

In [20]:
# generate code

start_index = random.randint(0, len(text) - MAX_SEQ_LENGTH - 1) # pick random code to start text generation

for diversity in [0.5, 1.0, 1.5]:
        print('-'*50, 'diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + MAX_SEQ_LENGTH]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, MAX_SEQ_LENGTH, VOCAB_SIZE))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

-------------------------------------------------- diversity: 0.5
----- Generating with seed: "tatic bool clone3_args_valid(struct kernel_clone_a"
tatic bool clone3_args_valid(struct kernel_clone_adiontadaeac
		 	 */tase_pata;
	snatct_crace_on_strscinnefaumsk-lice, sogn);

tase _uer =nsl_in_ged -dccsk_s aust * acus_sopsct>ceret, truck_tal_s_ bone_ */i	e, a	gerefina ==t0_samm_, lo <lisu->sintedef
tt  &set_ser)ou	isicuact_atre;t	rnse struc  saa_os_tnd * pcpge_csus_st a	ta
e  
		 eteo
 */
		truet tak_stact_pi atasaruct_structrseccomp.filter: therg ntie ser tl
 */a	 e A_ tocesoe tar e _pro Ap_sIre t teolp *i
			 e ine tre t->to_erff te   uatenc		=re argo_
	a	teid
	*/
			to = er
cn_ umm)s

use,iun tet (orgod {otr * Coto_olt ) freitata) ass i) eprc task ctute nt   ta*
l	 *    ;
 	no threr _sinnedcestrra tntcal fstode, ce_luso)

 *      o futch ie thitt ee s instre sthil er to me pread tas tl erotall a  tftee tsestors  o task anro ae eracathe fot  ce se  ursee er c  erien
	 * @forec ts nstant

In [21]:
# generate code

start_index = random.randint(0, len(text) - MAX_SEQ_LENGTH - 1) # pick random seed

for diversity in [0.5, 1.0, 1.5]:
        print('-'*50, 'diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + MAX_SEQ_LENGTH]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, MAX_SEQ_LENGTH, VOCAB_SIZE))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

-------------------------------------------------- diversity: 0.5
----- Generating with seed: " */
	if (match->notif)
		list_del(&n.list);
out:
	"
 */
	if (match->notif)
		list_del(&n.list);
out:
	sf (!tata) (
		sk_s =emm = "lritle __i (&ie __thar_astma)_not , in al i_er , 
i__sere t cerrers restcte)
{
	iftedard_pinti =Cs ilep_taeueon_coree
 dousee t, tnt srect ine bee an tere thate tin the malc n se ech(
t *oraterpe;
,
 *(intturtesiach cent weing 
		 */orsir(_sarler_sizi_not calatae;
	ifoff int__sriter= capae_d aeckr>tfk(>tsi_lenten > 
rar_eer alg c
		sizeota*&s_scrch
 *
 *
e p oueir( trer >ltritter *mrdinn sy _curct  ac th t ee sea dits roth biteat a shgin  seo cu stare stare of thee trinat  e 
al t of al tasc  of th st
 * @rol_adduseror ueda teteonetil    de tre re b o exr th .    uifcork tatesa that le ad to to iad ter
 *oiee _a dne ->roitren  = wqtki
 	n = proc_searomarscace_tas _serrental
	r	tue l= w_tiea_dun_ _

ditl f  OF	E__L_PAR;E		
nin_table_ ae __se );
_		curren__serec_ oee